### Required Imports

In [ ]:
import sys, nibabel as nib, numpy as np
sys.path.insert(0, 'core/')
from epi import  data_prep_ml
from utils import imask_ut
from denoiser import cnn
from sklearn.model_selection import train_test_split
from skimage.draw import circle
from skimage.transform import hough_circle, hough_circle_peaks
from keras.callbacks import ModelCheckpoint

### Set Path for Input Files

In [ ]:
ground_truth_ts = " " # Path of the ground truth in nifti format.
measured_fmri_ts = " " # Path of the extracted measured fMRI time series in nifti format. 
masks = " " # Path of the saved masks of the extracted slices. 

### Set Path for Output Files - Trained CNN

In [ ]:
Output = " "

### Preprocessing Before CNN Training

Needs no input if using the standard motion sequences preprogrammed for BrainDancer. If using custom sequences, provide the number of moving volumes to the data_prep_ml function. 

In [ ]:
measured_fmri = nib.load(measured_fmri_ts)
ground_truth = nib.load(ground_truth_ts)
imask = nib.load(masks)
imask_utils = imask_ut(imask)

In [ ]:
## stack_scn refers to the stack of measured fMRI time series
## stack_sim refers to the stack of ground truth time series
stack_scn, stack_sim, noise, stack_scn_flip, stack_sim_flip, noise_flip = data_prep_ml(ground_truth,measured_fmri,imask_utils,1,600)

### Set Training Parameters

In [ ]:
num_epochs = 250 # Enter the number of epochs for training

### Train CNN

In [ ]:
input_data = np.concatenate(((stack_scn/np.std(stack_scn,axis=1)[:,None]).reshape((len(stack_scn),600,1)),(stack_scn_flip/np.std(stack_scn,axis=1)[:,None]).reshape((len(stack_scn),600,1))))
output_data = np.concatenate(((stack_sim).reshape((len(stack_sim),600,1)),(stack_sim_flip).reshape((len(stack_sim),600,1))))
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.33, random_state=42)
ml = cnn(600)
filepath = Output + "/save_ml.h5"
mc = ModelCheckpoint(filepath, verbose=1, monitor= 'val_custom_loss',save_best_only=True)
history = ml.fit(X_train,y_train, nb_epoch = num_epochs, batch_size = 8,verbose=1, validation_data = (X_test,y_test),callbacks=[mc])